# Install Libraries

In [1]:
#Install the necessary libraries for forecasting
!pip install statsforecast mlforecast neuralforecast nixtla -q

Importing Libraries Required for this Notebook

In [2]:
!pip install xgboost -q

In [3]:
from nixtla import NixtlaClient
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsforecast import StatsForecast
from statsforecast.models import AutoTBATS, AutoARIMA, AutoETS
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from mlforecast.target_transforms import AutoDifferences
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT, NHITS
from neuralforecast.losses.pytorch import MAE
from nixtla import NixtlaClient
from functools import reduce
#import timesfm

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# Data Preparation

In [4]:
# data.columns

## Dropping the columns that wont be known in advance.

In [5]:
# data=data[["warehouse_y", "date", "orders", "holiday_y",	"shops_closed_y",	'winter_school_holidays_y',	"school_holidays_y"]]

In [6]:
# #rename as per requirement of the forecast libraries
# data = data.rename(columns={"warehouse_y": "unique_id", "date": "ds", "orders": "y","holiday_y":"holiday", "shops_closed_y":"shops_closed",
#                             'winter_school_holidays_y':"winter_school_holidays", "school_holidays_y":"school_holidays"})

In [7]:
# data.head()

In [8]:
# data.info()

In [9]:
# data['ds'] = pd.to_datetime(data['ds'])

In [10]:
# data["unique_id"].value_counts()

In [11]:
# prompt: delete all uniqye_id ds duplicate columns

In [12]:
# data.groupby('unique_id').count()

In [13]:
# #Splitting the Data for Train & Test
# train = data[data['ds'] <= '2024-01-15']
# test = data[data['ds'] > '2024-01-15']

In [14]:
# train.groupby('unique_id').count()

In [15]:
# train.head()

In [16]:
train=pd.read_csv("Train_df_NueralForecast.csv")
test_=pd.read_csv("Future_df_NueralForecast.csv")

In [17]:
train['ds'] = pd.to_datetime(train['ds'])
test_['ds'] = pd.to_datetime(test_['ds'])

# SFFORECAST

###Defining Statforecast Parameters

In [18]:
models = [AutoARIMA(season_length=365)]

sf = StatsForecast(models=models, freq='D', n_jobs=1)

In [19]:
test_

,Unnamed: 0,unique_id,ds,holiday,shops_closed,winter_school_holidays,school_holidays,year,month,day,day_of_week,week_of_year,shops_closed_lag,Shop_opened_after_close
0,1133,Brno_1,2024-01-16,0,0,0,0,2024,1,16,1,3,0.0,0
1,1134,Brno_1,2024-01-17,0,0,0,0,2024,1,17,2,3,0.0,0
2,1135,Brno_1,2024-01-18,0,0,0,0,2024,1,18,3,3,0.0,0
3,1136,Brno_1,2024-01-19,0,0,0,0,2024,1,19,4,3,0.0,0
4,1137,Brno_1,2024-01-20,0,0,0,0,2024,1,20,5,3,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1188,Prague_3,2024-03-11,0,0,0,0,2024,3,11,0,11,0.0,0
236,1189,Prague_3,2024-03-12,0,0,0,0,2024,3,12,1,11,0.0,0
237,1190,Prague_3,2024-03-13,0,0,0,0,2024,3,13,2,11,0.0,0
238,1191,Prague_3,2024-03-14,0,0,0,0,2024,3,14,3,11,0.0,0


In [20]:
test_.groupby('unique_id').count()

,Unnamed: 0,ds,holiday,shops_closed,winter_school_holidays,school_holidays,year,month,day,day_of_week,week_of_year,shops_closed_lag,Shop_opened_after_close
unique_id,,,,,,,,,,,,,
Brno_1,60,60,60,60,60,60,60,60,60,60,60,60,60
Prague_1,60,60,60,60,60,60,60,60,60,60,60,60,60
Prague_2,60,60,60,60,60,60,60,60,60,60,60,60,60
Prague_3,60,60,60,60,60,60,60,60,60,60,60,60,60


In [21]:
train=train[['unique_id', 'ds', 'y', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays']]
test_=test_[['unique_id', 'ds', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays']]

In [ ]:
horizon = 60
sf_fcst = sf.forecast(df=train, h=horizon, X_df=test_, level=(5,10,20,30,40,50,60,70,80,90,95))

In [ ]:
sf_fcst.head()

### Defining AutoTBATS & AutoETS

In [ ]:
models = [AutoTBATS(season_length=365),AutoETS(season_length=365)]

NameError: name 'AutoTBATS' is not defined

In [ ]:
stat_fcst = StatsForecast(models=models,freq='D')

In [ ]:
train_stat_df_ETS_TBATS = train[["unique_id", "ds", "y"]]

NameError: name 'train' is not defined

In [ ]:
ets_tbats = stat_fcst.forecast(df=train_stat_df_ETS_TBATS, h=60, level=(5,10,20,30,40,50,60,70,80,90,95))

NameError: name 'train_stat_df_ETS_TBATS' is not defined

In [ ]:
ets_tbats.head()

In [ ]:
# # @title ds vs AutoARIMA

# from matplotlib import pyplot as plt
# import seaborn as sns
# def _plot_series(series, series_name, series_index=0):
#   palette = list(sns.palettes.mpl_palette('Dark2'))
#   xs = series['ds']
#   ys = series['AutoARIMA']

#   # plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

# fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
# df_sorted = sf_fcst.sort_values('ds', ascending=True)
# for i, (series_name, series) in enumerate(df_sorted.groupby('unique_id')):
#   _plot_series(series, series_name, i)
#   fig.legend(title='unique_id', bbox_to_anchor=(1, 1), loc='upper left')
# sns.despine(fig=fig, ax=ax)
# plt.xlabel('ds')
# _ = plt.ylabel('AutoARIMA')

# MLForecast

In [ ]:
# sf_fcst.to_csv("sf_forecast_weekly_merged.csv")
# ets_tbats.to_csv("ets_tbats_forecast_weekly_merged.csv")

In [ ]:

# from google.colab import files
# files.download("sf_forecast_weekly_merged.csv")
# files.download("ets_tbats_forecast_weekly_merged.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
models = [LinearRegression(),
    lgb.LGBMRegressor(verbosity=-1),
    xgb.XGBRegressor(),
    RandomForestRegressor(random_state=0),
]
@njit
def rolling_mean_7(x):
    return rolling_mean(x, window_size=7)
@njit
def rolling_mean_28(x):
    return rolling_mean(x, window_size=28)

mlf = MLForecast(
    models=models,
    freq='D',
    lags=[7, 14, 28, 90, 180],

    lag_transforms={
        1: [expanding_mean],
       7: [rolling_mean_7, rolling_mean_28],
        14: [rolling_mean_7, rolling_mean_28],
        28: [rolling_mean_7, rolling_mean_28],
        90: [rolling_mean_7, rolling_mean_28],
    },
    date_features=['year', 'month', 'day', 'quarter'],
    target_transforms=[AutoDifferences(max_diffs=2)])


In [ ]:
from mlforecast.utils import PredictionIntervals

In [ ]:
train

,ds,unique_id,y,holiday,shops_closed,winter_school_holidays,school_holidays
0,2020-12-06,Prague_1,13479,0,0,0,0
1,2020-12-13,Prague_1,49517,0,0,0,0
2,2020-12-20,Prague_1,53288,0,0,0,0
3,2020-12-27,Prague_1,45261,2,1,0,0
4,2021-01-03,Prague_1,45811,1,1,0,0
...,...,...,...,...,...,...,...
639,2023-04-16,Brno_1,51340,1,1,0,0
640,2023-04-23,Brno_1,51150,0,0,0,0
641,2023-04-30,Brno_1,51683,0,0,0,0
642,2023-05-07,Brno_1,50302,1,0,0,0


In [ ]:
mlf.fit(
    train,
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    static_features=[],
    prediction_intervals=PredictionIntervals(n_windows=2, h=44),
)

MLForecast(models=[LinearRegression, LGBMRegressor, XGBRegressor, RandomForestRegressor], freq=W, lag_features=['lag1', 'lag4', 'lag13', 'lag26', 'expanding_mean_lag1', 'rolling_mean_4_lag4', 'rolling_mean_13_lag4', 'rolling_mean_4_lag13', 'rolling_mean_13_lag13'], date_features=['year', 'month', 'day', 'quarter'], num_threads=1)

In [ ]:
#For using in predict function
future_df = test[["unique_id", "ds", 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']]

In [ ]:
levels=[5,10,20,30,40,50,60,70,80,90,95]

In [ ]:
mlforecast = mlf.predict(h=60, X_df=future_df, level=levels)

In [ ]:
mlforecast.head()

,unique_id,ds,LinearRegression,LGBMRegressor,XGBRegressor,RandomForestRegressor,LinearRegression-lo-95,LinearRegression-lo-90,LinearRegression-lo-80,LinearRegression-lo-70,...,RandomForestRegressor-hi-10,RandomForestRegressor-hi-20,RandomForestRegressor-hi-30,RandomForestRegressor-hi-40,RandomForestRegressor-hi-50,RandomForestRegressor-hi-60,RandomForestRegressor-hi-70,RandomForestRegressor-hi-80,RandomForestRegressor-hi-90,RandomForestRegressor-hi-95
0,Brno_1,2023-05-21,52761.230469,52547.792969,52520.937500,52956.500000,51927.864258,51994.470703,52127.683594,52260.896484,...,52972.109375,52987.718750,53003.328125,55575.134375,59425.039062,63274.943750,67124.848437,70974.753125,74824.657812,76749.610156
1,Brno_1,2023-05-28,52618.945312,52336.414062,52851.566406,52840.949219,52004.489844,52029.764844,52080.314844,52130.864844,...,53863.147656,54885.346094,55907.544531,61060.057031,68277.726562,75495.396094,82713.065625,89930.735156,97148.404687,100757.239453
2,Brno_1,2023-06-04,53323.253906,52806.445312,52488.191406,52455.300781,49289.784766,49504.253125,49933.189844,50362.126563,...,53556.337109,54657.373437,55758.409766,63755.471484,75200.545898,86645.620313,98090.694727,109535.769141,120980.843555,126703.380762
3,Brno_1,2023-06-11,53038.941406,53225.093750,53569.750000,53192.121094,45954.689551,46501.652539,47595.578516,48689.504492,...,54760.469531,56328.817969,57897.166406,68683.686719,84079.292969,99474.899219,114870.505469,130266.111719,145661.717969,153359.521094
4,Brno_1,2023-06-18,53456.707031,54206.015625,54576.636719,53482.449219,43076.983691,43610.354102,44677.094922,45743.835742,...,54818.343359,56154.237500,57490.131641,71066.164453,90762.266602,110458.368750,130154.470898,149850.573047,169546.675195,179394.726270


# Neural Forecast

In [ ]:
hist_exog_list = ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']
levels = [5,10,20,30,40,50,60,70,80,90,95]
horizon = 60  # Number of future time steps to predict
from neuralforecast.losses.pytorch import MQLoss
# Initialize the TFT model
tft_model = TFT(h=60,
                input_size=120,
                hist_exog_list=hist_exog_list,loss=MQLoss(level=levels),
                max_steps=200,
                start_padding_enabled=True)

# Initialize the NHITS model
nhits_model = NHITS(h=60,
                input_size=120,
                hist_exog_list=hist_exog_list,loss=MQLoss(level=levels),
                max_steps=200,
                start_padding_enabled=True)

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
nf = NeuralForecast(models=[tft_model, nhits_model], freq='D')

nf.fit(df=train)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MQLoss                   | 23     | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 1.5 K  | train
4 | temporal_encoder        | TemporalCovariateEncoder | 948 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 256 K  | train
6 | output_adapter          | Linear      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=200` reached.


In [ ]:
neural_fcst = nf.predict(futr_df = future_df)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
neural_fcst.head()

,ds,TFT-median,TFT-lo-90,TFT-lo-80,TFT-hi-80,TFT-hi-90,NHITS-median,NHITS-lo-95,NHITS-lo-90,NHITS-lo-80,...,NHITS-hi-10,NHITS-hi-20,NHITS-hi-30,NHITS-hi-40,NHITS-hi-50,NHITS-hi-60,NHITS-hi-70,NHITS-hi-80,NHITS-hi-90,NHITS-hi-95
unique_id,,,,,,,,,,,,,,,,,,,,,
Brno_1,2023-05-21,52925.574219,51234.234375,51649.609375,54314.937500,54552.761719,53760.566406,51862.371094,52378.117188,52671.707031,...,53855.820312,54332.800781,54395.445312,54298.894531,54662.960938,54997.558594,55560.191406,55795.566406,56685.027344,57296.660156
Brno_1,2023-05-28,52744.039062,51077.406250,51457.949219,54166.609375,54433.960938,53600.570312,51323.839844,52010.152344,52594.628906,...,53832.800781,54347.289062,54504.457031,54594.003906,55090.718750,55627.015625,55976.281250,56021.636719,56758.843750,57119.945312
Brno_1,2023-06-04,52723.171875,51025.027344,51409.093750,54182.710938,54455.675781,54153.839844,51620.746094,51965.328125,52614.503906,...,54747.953125,54632.175781,54719.363281,55100.210938,55366.093750,55750.398438,55992.246094,56719.761719,57393.500000,57818.449219
Brno_1,2023-06-11,52609.726562,50982.226562,51346.832031,54059.718750,54345.437500,54355.089844,51519.289062,52103.785156,52900.339844,...,54990.398438,54902.015625,55083.664062,55436.359375,55585.218750,55887.632812,55998.449219,56729.179688,57185.273438,57328.484375
Brno_1,2023-06-18,52437.492188,50979.894531,51311.925781,53817.082031,54107.531250,54649.976562,52091.582031,52613.328125,52975.484375,...,55289.128906,55245.253906,55186.324219,55629.234375,55724.789062,56067.175781,56374.500000,56995.167969,57351.082031,58145.433594


In [ ]:

neural_fcst.to_csv("neural_forecast_weekly_merged.csv")

In [ ]:

from google.colab import files

files.download("neural_forecast_weekly_merged.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# TimeGPT

In [ ]:

# Initialize the NixtlaClient
nixtla_client = NixtlaClient(api_key='nixak-dZyIsJpKaqa6jjcPkhMCcHNI6S9heGWdO5iG2WixOCViM5afbymNqIRgE1XZHgkTx7MhzC9lhzFz4eKm')

# Assuming df is your input dataframe with historical data
# and future_df is your dataframe with future exogenous variables

# Specify the exogenous variables
exog_vars = ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']
# Make the forecast
timegpt_fcst_zero = nixtla_client.forecast(df=train, h=60, X_df=future_df, freq='D', level=(5,10,20,30,40,50,60,70,80,90,95))


In [ ]:
timegpt_fcst_zero['ds'] = pd.to_datetime(timegpt_fcst_zero['ds'])

In [ ]:
timegpt_fcst_zero.head()

,unique_id,ds,TimeGPT,TimeGPT-hi-10,TimeGPT-hi-20,TimeGPT-hi-30,TimeGPT-hi-40,TimeGPT-hi-5,TimeGPT-hi-50,TimeGPT-hi-60,...,TimeGPT-lo-20,TimeGPT-lo-30,TimeGPT-lo-40,TimeGPT-lo-5,TimeGPT-lo-50,TimeGPT-lo-60,TimeGPT-lo-70,TimeGPT-lo-80,TimeGPT-lo-90,TimeGPT-lo-95
0,Brno_1,2023-05-21,50947.640,51064.348,51179.760,51598.484,51630.613,51041.710,52159.880,52785.008,...,50715.520,50296.797,50264.668,50853.570,49735.402,49110.273,48768.773,48609.207,48352.297,48343.777
1,Brno_1,2023-05-28,50373.836,50975.100,51178.605,51385.230,51462.010,50814.895,52193.970,52564.120,...,49569.066,49362.440,49285.660,49932.777,48553.703,48183.550,47873.960,47588.652,47462.938,47187.260
2,Brno_1,2023-06-04,50199.540,51030.780,51623.450,52321.617,52401.227,50714.188,52419.246,53587.504,...,48775.630,48077.460,47997.850,49684.890,47979.832,46811.574,46031.027,45937.625,45610.246,45270.850
3,Brno_1,2023-06-11,50015.566,50286.242,50609.113,50628.266,50703.490,50099.547,50977.492,51579.324,...,49422.020,49402.867,49327.645,49931.586,49053.640,48451.810,47045.047,45713.050,45564.188,45347.242
4,Brno_1,2023-06-18,49981.720,50468.690,51029.082,51481.043,52208.836,50185.973,52793.062,53044.773,...,48934.355,48482.395,47754.600,49777.465,47170.375,46918.664,46099.168,45073.620,44845.730,44818.910


## Timesfm

In [ ]:
# Y_test_df = test.copy()
# df = train.copy()

In [ ]:
# tfm = timesfm.TimesFm(
#     context_len=512,
#     horizon_len=8,
#     input_patch_len=32,
#     output_patch_len=128,
#     num_layers=20,
#     model_dims=1280,
#     backend="cpu",
# )
# tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

In [ ]:
# timesfm_df = tfm.forecast_on_df(
#     inputs=df,
#     freq='MS',
#     value_name='y',
#     num_jobs=-1,
# )

In [ ]:
timesfm_df = pd.read_csv("timesfm_weekly.csv")

In [ ]:
timesfm_df['ds'] = pd.to_datetime(timesfm_df['ds'])

In [ ]:
timegpt_fcst_zero.head()

,unique_id,ds,TimeGPT,TimeGPT-hi-10,TimeGPT-hi-20,TimeGPT-hi-30,TimeGPT-hi-40,TimeGPT-hi-5,TimeGPT-hi-50,TimeGPT-hi-60,...,TimeGPT-lo-20,TimeGPT-lo-30,TimeGPT-lo-40,TimeGPT-lo-5,TimeGPT-lo-50,TimeGPT-lo-60,TimeGPT-lo-70,TimeGPT-lo-80,TimeGPT-lo-90,TimeGPT-lo-95
0,Brno_1,2023-05-21,50947.640,51064.348,51179.760,51598.484,51630.613,51041.710,52159.880,52785.008,...,50715.520,50296.797,50264.668,50853.570,49735.402,49110.273,48768.773,48609.207,48352.297,48343.777
1,Brno_1,2023-05-28,50373.836,50975.100,51178.605,51385.230,51462.010,50814.895,52193.970,52564.120,...,49569.066,49362.440,49285.660,49932.777,48553.703,48183.550,47873.960,47588.652,47462.938,47187.260
2,Brno_1,2023-06-04,50199.540,51030.780,51623.450,52321.617,52401.227,50714.188,52419.246,53587.504,...,48775.630,48077.460,47997.850,49684.890,47979.832,46811.574,46031.027,45937.625,45610.246,45270.850
3,Brno_1,2023-06-11,50015.566,50286.242,50609.113,50628.266,50703.490,50099.547,50977.492,51579.324,...,49422.020,49402.867,49327.645,49931.586,49053.640,48451.810,47045.047,45713.050,45564.188,45347.242
4,Brno_1,2023-06-18,49981.720,50468.690,51029.082,51481.043,52208.836,50185.973,52793.062,53044.773,...,48934.355,48482.395,47754.600,49777.465,47170.375,46918.664,46099.168,45073.620,44845.730,44818.910


In [ ]:
# Make the forecast with timegpt finetuned
timegpt_fcst_finetune_4 = nixtla_client.forecast(df=train, h=60, X_df=future_df, freq='D', finetune_steps=4, level=(5,10,20,30,40,50,60,70,80,90,95))

In [ ]:
timegpt_fcst_finetune_6 = nixtla_client.forecast(df=train, h=60, X_df=future_df, freq='D', finetune_steps=6, level=(5,10,20,30,40,50,60,70,80,90,95))

In [ ]:
timegpt_fcst_finetune_8= nixtla_client.forecast(df=train, h=60, X_df=future_df, freq='D', finetune_steps=8, level=(5,10,20,30,40,50,60,70,80,90,95))

In [ ]:
timegpt_fcst_finetune_6.rename(columns={'TimeGPT': 'TimeGPT_6'}, inplace=True)

In [ ]:
timegpt_fcst_finetune_4.rename(columns={'TimeGPT': 'TimeGPT_4'}, inplace=True)

In [ ]:
timegpt_fcst_finetune_8.rename(columns={'TimeGPT': 'TimeGPT_8'}, inplace=True)

In [ ]:
timegpt_fcst_finetune_4.columns = ['TimeGPT_4-' + str(col) if 'TimeGPT-' in str(col) else col for col in timegpt_fcst_finetune_4.columns]
timegpt_fcst_finetune_6.columns = ['TimeGPT_6-' + str(col) if 'TimeGPT-' in str(col) else col for col in timegpt_fcst_finetune_6.columns]






In [ ]:


timegpt_fcst_finetune_8.columns = ['TimeGPT_8-' + str(col) if 'TimeGPT-' in str(col) else col for col in timegpt_fcst_finetune_8.columns]

In [ ]:
timesfm_df = pd.read_csv("timesfm_weekly.csv")

In [ ]:

timesfm_df['ds'] = pd.to_datetime(timesfm_df['ds'])

In [ ]:
#Merge All Results
data_frames = [sf_fcst, ets_tbats,mlforecast, neural_fcst,timegpt_fcst_zero,timegpt_fcst_finetune_4, timegpt_fcst_finetune_6, timegpt_fcst_finetune_8, timesfm_df ]
final_forecast_monthly_merged = reduce(lambda  left,right: pd.merge(left,right,on=['unique_id', 'ds'],how='inner'), data_frames)

In [ ]:
final_forecast_monthly_merged.to_csv("final_forecast_weekly_merged.csv")

In [ ]:
# prompt: write code to download final_forecast_weekly_merged.csv to local

from google.colab import files
files.download('final_forecast_weekly_merged.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>